In [ ]:
# Run CPU-only, GPU code needs further testing
"""
Restart kernel after running
Only need to run once
"""
!pip install scikit-learn matplotlib seaborn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import torch
import torch.utils.data as td
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, Subset
import wandb


froot = './data/k562_samp_epft_norm_test_1.csv'

df = pd.read_csv(froot)

/grid/it/data/elzar/easybuild/software/Anaconda3/2020.02/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (2.0.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/grid/siepel/home_norepl/hassett/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
print(df.head())

  seqnames     start       end strand  ensembl_gene_id  score      ctcf  \
0       15  88623545  88623545      +  ENSG00000181026    0.0 -0.079992   
1       15  88623546  88623546      +  ENSG00000181026    0.0 -0.079942   
2       15  88623547  88623547      +  ENSG00000181026    0.0 -0.079893   
3       15  88623548  88623548      +  ENSG00000181026    0.0 -0.079844   
4       15  88623549  88623549      +  ENSG00000181026    0.0 -0.079796   

   h3k36me3   h3k4me1  h3k79me2   h3k9me1   h3k9me3  h4k20me1       sj5  \
0 -0.000099  0.348531  4.423451  0.446508 -0.168099  3.232475 -0.028916   
1  0.001638  0.352677  4.460072  0.453024 -0.169218  3.259194 -0.028916   
2  0.003360  0.356807  4.496664  0.459491 -0.170339  3.285849 -0.028916   
3  0.005065  0.360919  4.533223  0.465908 -0.171461  3.312435 -0.028916   
4  0.006754  0.365013  4.569743  0.472274 -0.172584  3.338952 -0.028916   

        sj3       dms  wgbs      rpts  lambda_alphaj      zeta  
0 -0.057178 -0.307549   0.0  0.24

In [3]:
column_names = np.array(df.columns)
feature_names = column_names[7:12]#column_names[6:-2]
num_features = len(feature_names)
#nucleotides = column_names[-6:-2]
print(feature_names)
#print(nucleotides)
num_samples = df.shape[0]

# process read counts
X_ji = df['score'].values

# process GLM simulated elongation rates
Z_ji = df['zeta'].values

['h3k36me3' 'h3k4me1' 'h3k79me2' 'h3k9me1' 'h3k9me3']


In [4]:
print("Number of Samples: " + str(num_samples))

Number of Samples: 16182613


In [5]:
print("Number of Features: " + str(num_features))

Number of Features: 5


In [6]:
#Y_ji is a list of samples containing lists of their feature values
    # [   
    #   sample_1: [feat_1, feat_2,...,feat_n],
    #   sample_2: [feat_1, feat_2,...,feat_n],
    # ]

Y_ji = df.iloc[:, 6:-2].values
Y_ji_shape = Y_ji.shape
print(Y_ji.shape)

(16182613, 12)


In [7]:
C_j = df['lambda_alphaj'].values

In [8]:
gene_ids = df['ensembl_gene_id'].values

In [9]:
print("Number of genes: " + str(len(np.unique(gene_ids))))

Number of genes: 519


In [10]:
cuda_available = torch.cuda.is_available()
print("CUDA (GPU support) is available:", cuda_available)
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

CUDA (GPU support) is available: False
Number of GPUs available: 0


In [24]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from pdb import set_trace as stop

# AttentiveChrome Model
def batch_product(iput, mat2):
    result = None
    for i in range(iput.size()[0]):
        op = torch.mm(iput[i], mat2)
        op = op.unsqueeze(0)
        if(result is None):
            result = op
        else:
            result = torch.cat((result,op),0)
    return result.squeeze(2)


class rec_attention(nn.Module):
    # attention with bin context vector per HM and HM context vector
    def __init__(self,hm,args):
        super(rec_attention,self).__init__()
        self.num_directions=2 if args.bidirectional else 1
        if (hm==False):
            self.bin_rep_size=args.bin_rnn_size*self.num_directions
        else:
            self.bin_rep_size=args.bin_rnn_size

        self.bin_context_vector=nn.Parameter(torch.Tensor(self.bin_rep_size,1),requires_grad=True)


        self.softmax=nn.Softmax(dim=1)

        self.bin_context_vector.data.uniform_(-0.1, 0.1)

    def forward(self,iput):
        alpha=self.softmax(batch_product(iput,self.bin_context_vector))
        [batch_size,source_length,bin_rep_size2]=iput.size()
        repres=torch.bmm(alpha.unsqueeze(2).view(batch_size,-1,source_length),iput)
        return repres,alpha



class recurrent_encoder(nn.Module):
    # modular LSTM encoder
    def __init__(self,n_bins,ip_bin_size,hm,args):
        super(recurrent_encoder,self).__init__()
        self.bin_rnn_size=args.bin_rnn_size
        self.ipsize=ip_bin_size
        self.seq_length=n_bins

        self.num_directions=2 if args.bidirectional else 1
        if (hm==False):
            self.bin_rnn_size=args.bin_rnn_size
        else:
            self.bin_rnn_size=args.bin_rnn_size // 2
        self.bin_rep_size=self.bin_rnn_size*self.num_directions


        self.rnn=nn.LSTM(self.ipsize,self.bin_rnn_size,num_layers=args.num_layers,dropout=args.dropout,bidirectional=args.bidirectional)

        self.bin_attention=rec_attention(hm,args)
    def outputlength(self):
        return self.bin_rep_size
    def forward(self,single_hm,hidden=None):
        bin_output, hidden = self.rnn(single_hm,hidden)
        bin_output = bin_output.permute(1,0,2)
        hm_rep,bin_alpha = self.bin_attention(bin_output)
        return hm_rep,bin_alpha


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self


class att_chrome(nn.Module):
    def __init__(self,args):
        super(att_chrome,self).__init__()
        self.n_hms=args.n_hms
        self.n_bins=args.n_bins
        self.ip_bin_size=1

        self.rnn_hms=nn.ModuleList()
        for i in range(self.n_hms):
            self.rnn_hms.append(recurrent_encoder(self.n_bins,self.ip_bin_size,False,args))
        self.opsize = self.rnn_hms[0].outputlength()
        self.hm_level_rnn_1=recurrent_encoder(self.n_hms,self.opsize,True,args)
        self.opsize2=self.hm_level_rnn_1.outputlength()
        self.diffopsize=2*(self.opsize2)
        self.fdiff1_1=nn.Linear(self.opsize2,100)

    def forward(self,iput):

        bin_a=None
        level1_rep=None
        [batch_size,_,_]=iput.size()

        for hm,hm_encdr in enumerate(self.rnn_hms):
            hmod=iput[:,:,hm].contiguous()
            hmod=torch.t(hmod).unsqueeze(2)

            op,a= hm_encdr(hmod)
            if level1_rep is None:
                level1_rep=op
                bin_a=a
            else:
                level1_rep=torch.cat((level1_rep,op),1)
                bin_a=torch.cat((bin_a,a),1)
        level1_rep=level1_rep.permute(1,0,2)
        final_rep_1,hm_level_attention_1=self.hm_level_rnn_1(level1_rep)
        final_rep_1=final_rep_1.squeeze(1)
        prediction_m=((self.fdiff1_1(final_rep_1)))

        return prediction_m

# dropout won't be applied to single-layer RNN
args_dict = {'lr': 0.0001, 'model_name': 'attchrome', 'clip': 1, 'epochs': 2, 'batch_size': 10, 'dropout': 0.5, 'cell_1': 'Cell1', 'save_root': 'Results/Cell1', 'data_root': 'data/', 'gpuid': 0, 'gpu': 0, 'n_hms': 5, 'n_bins': 200, 'bin_rnn_size': 32, 'num_layers': 1, 'unidirectional': False, 'save_attention_maps': False, 'attentionfilename': 'beta_attention.txt', 'test_on_saved_model': False, 'bidirectional': True, 'dataset': 'Cell1'}
att_chrome_args = AttrDict(args_dict)
att_chrome_model = att_chrome(att_chrome_args)

dtype = torch.FloatTensor

/grid/siepel/home_norepl/hassett/.local/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [25]:
print("==>initializing a new model")
for p in att_chrome_model.parameters():
    p.data.uniform_(-0.1,0.1)

optimizer = optim.Adam(att_chrome_model.parameters(), lr = 0.0002)
#optimizer = optim.SGD(model.parameters(), lr = args.lr, momentum=args.momentum)

==>initializing a new model


In [26]:
"""
# original code should also work for grouping data
import collections

count = 0
attr=collections.OrderedDict()
windows = 100

# need to split by gene later
for i in range(0,num_samples,windows):
        hm1=torch.zeros(windows,1)
        hm2=torch.zeros(windows,1)
        hm3=torch.zeros(windows,1)
        hm4=torch.zeros(windows,1)
        hm5=torch.zeros(windows,1)
        x_ji=torch.zeros(windows,1)
        c_j=torch.zeros(windows,1)
        z_ji=torch.zeros(windows,1)
        for w in range(0,windows):
            hm1[w][0]=int(Y_ji[i+w][0])
            hm2[w][0]=int(Y_ji[i+w][1])
            hm3[w][0]=int(Y_ji[i+w][2])
            hm4[w][0]=int(Y_ji[i+w][3])
            hm5[w][0]=int(Y_ji[i+w][4])
            x_ji[w][0]=int(X_ji[i+w])
            c_j[w][0]=int(C_j[i+w])
            z_ji[w][0]=int(Z_ji[i+w])
        geneID=gene_ids[i]

        attr[count]={
            'geneID':geneID,
            'hm1':hm1,
            'hm2':hm2,
            'hm3':hm3,
            'hm4':hm4,
            'hm5':hm5,
            'x_ji':x_ji,
            'c_j':c_j,
            'z_ji':z_ji
        }
        count+=1
"""

"\n# original code should also work for grouping data\nimport collections\n\ncount = 0\nattr=collections.OrderedDict()\nwindows = 100\n\n# need to split by gene later\nfor i in range(0,num_samples,windows):\n        hm1=torch.zeros(windows,1)\n        hm2=torch.zeros(windows,1)\n        hm3=torch.zeros(windows,1)\n        hm4=torch.zeros(windows,1)\n        hm5=torch.zeros(windows,1)\n        x_ji=torch.zeros(windows,1)\n        c_j=torch.zeros(windows,1)\n        z_ji=torch.zeros(windows,1)\n        for w in range(0,windows):\n            hm1[w][0]=int(Y_ji[i+w][0])\n            hm2[w][0]=int(Y_ji[i+w][1])\n            hm3[w][0]=int(Y_ji[i+w][2])\n            hm4[w][0]=int(Y_ji[i+w][3])\n            hm5[w][0]=int(Y_ji[i+w][4])\n            x_ji[w][0]=int(X_ji[i+w])\n            c_j[w][0]=int(C_j[i+w])\n            z_ji[w][0]=int(Z_ji[i+w])\n        geneID=gene_ids[i]\n\n        attr[count]={\n            'geneID':geneID,\n            'hm1':hm1,\n            'hm2':hm2,\n            'hm

In [27]:
class GeneDataset(Dataset):
    def __init__(self, GeneId, Y_ji, X_ji, C_j, Z_ji):
        self.GeneId = GeneId
        self.Y_ji = Y_ji
        self.X_ji = X_ji
        self.C_j = C_j
        self.Z_ji = Z_ji

    def __len__(self):
        # Only count full batches
        return len(self.Y_ji) // 100

    def __getitem__(self, idx):
        batch_start = idx * 100
        batch_end = batch_start + 100

        data = {
            #'GeneId':  self.GeneId[batch_start:batch_end], # Uncomment if needed
            'Y_ji':  torch.tensor(self.Y_ji[batch_start:batch_end], dtype=torch.float32),
            'X_ji': torch.tensor(self.X_ji[batch_start:batch_end], dtype=torch.float32),
            'C_j': torch.tensor(self.C_j[batch_start:batch_end], dtype=torch.float32),
            'Z_ji': torch.tensor(self.Z_ji[batch_start:batch_end], dtype=torch.float32)
        }
        return data

In [28]:
from sklearn.model_selection import train_test_split

train_gene_ids, test_gene_ids, train_Y_ji, test_Y_ji, train_X_ji, test_X_ji, train_C_j, test_C_j, train_Z_ji, test_Z_ji = train_test_split(gene_ids, Y_ji, X_ji, C_j, Z_ji, test_size=0.2, shuffle=False)

# Create dataset instances for train and test
train_dataset = GeneDataset(train_gene_ids, train_Y_ji, train_X_ji, train_C_j, train_Z_ji)
test_dataset = GeneDataset(test_gene_ids, test_Y_ji, test_X_ji, test_C_j, test_Z_ji)

In [29]:
"""
class HMData(Dataset):
    # Dataset class for loading data
    def __init__(self,data_cell1,transform=None):
        self.c1=data_cell1
    def __len__(self):
        return len(self.c1)
    def __getitem__(self,i):
        x_ji=self.c1[i]['x_ji']
        c_j=self.c1[i]['c_j']
        z_ji=self.c1[i]['z_ji']
        geneID=self.c1[i]['geneID']
        sample={'geneID':geneID,
               'input':final_data_c1,
               'x_ji':x_ji,
                'c_j':c_j,
                'z_ji':z_ji
               }
        return sample
"""

"\nclass HMData(Dataset):\n    # Dataset class for loading data\n    def __init__(self,data_cell1,transform=None):\n        self.c1=data_cell1\n    def __len__(self):\n        return len(self.c1)\n    def __getitem__(self,i):\n        x_ji=self.c1[i]['x_ji']\n        c_j=self.c1[i]['c_j']\n        z_ji=self.c1[i]['z_ji']\n        geneID=self.c1[i]['geneID']\n        sample={'geneID':geneID,\n               'input':final_data_c1,\n               'x_ji':x_ji,\n                'c_j':c_j,\n                'z_ji':z_ji\n               }\n        return sample\n"

In [30]:
"""
from torch.utils.data import random_split

hm_dataset=HMData(attr)
train_size = int(0.8 * len(hm_dataset))
test_size = len(hm_dataset) - train_size
train_dataset, test_dataset = random_split(hm_dataset, [train_size, test_size])
"""

Train = torch.utils.data.DataLoader(train_dataset, batch_size=500, shuffle=True)
#Valid = torch.utils.data.DataLoader(valid_inputs, batch_size=args.batch_size, shuffle=False)
Test = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [31]:
print(len(Train))

259


In [32]:
"""
def build_dataset(train_data, batch_size):
    batches = create_batches(train_data.groupby('GeneId'), batch_size)
    dataset = GeneDataset(batches)
    loader = DataLoader(dataset, batch_size=1, num_workers=7, shuffle=False, pin_memory=True)
    return loader
"""

"\ndef build_dataset(train_data, batch_size):\n    batches = create_batches(train_data.groupby('GeneId'), batch_size)\n    dataset = GeneDataset(batches)\n    loader = DataLoader(dataset, batch_size=1, num_workers=7, shuffle=False, pin_memory=True)\n    return loader\n"

In [33]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()

    def forward(self, X_ji, C_j, rho_ji):
        C_j_value = C_j[0]
        #print(X_ji.shape)
        #print(rho_ji.shape)
        X_ji = X_ji.squeeze(0)
        rho_ji = rho_ji.squeeze(0).squeeze(1)
        loss = X_ji * rho_ji + C_j_value * torch.exp(-rho_ji) - X_ji * torch.log(C_j_value)
        return (loss).mean()

In [34]:
"""
def build_optimizer(network, optimizer, learning_rate, momentum):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=momentum)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer
"""

'\ndef build_optimizer(network, optimizer, learning_rate, momentum):\n    if optimizer == "sgd":\n        optimizer = optim.SGD(network.parameters(),\n                              lr=learning_rate, momentum=momentum)\n    elif optimizer == "adam":\n        optimizer = optim.Adam(network.parameters(),\n                               lr=learning_rate)\n    return optimizer\n'

In [35]:
def train_epoch(model, loader, optimizer, loss_fn):
    model.train()
    print(len(loader))
    total_loss = 0
    for idx, batch in enumerate(loader):
        optimizer.zero_grad()
        Y_ji_batch = batch['Y_ji'].to(device)
        X_ji_batch = batch['X_ji'].to(device)
        C_j_batch = batch['C_j'].to(device)
        outputs = model(Y_ji_batch)
        loss = loss_fn(X_ji_batch, C_j_batch, outputs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    return avg_loss

In [ ]:
def print_profiler_results(profiler):
    print(profiler.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))

loader = Train#build_dataset(train_data, 16)#16000000)
optimizer = optimizer#build_optimizer(model, "adam", 1e-4, 0)
loss_fn = CustomLoss()

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3),
    on_trace_ready=print_profiler_results,
    record_shapes=True,
    profile_memory=True
) as profiler:
    for epoch in range(3):
        print(f'Epoch {epoch+1}')
        att_chrome_model.train()
        print(len(loader))
        total_loss = 0
        for idx, batch in enumerate(loader):
            optimizer.zero_grad()
            Y_ji_batch = batch['Y_ji']#.to(device)
            X_ji_batch = batch['X_ji']#.to(device)
            C_j_batch = batch['C_j']#.to(device)
            outputs = att_chrome_model(Y_ji_batch)
            loss = loss_fn(X_ji_batch, C_j_batch, outputs)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(att_chrome_model.parameters(), 1)
            optimizer.step()
            total_loss += loss.item()
            #profiler.step()
        print("total")
        avg_loss = total_loss / len(loader)
        print(avg_loss)

Epoch 1
259
total
0.11065784364248335
Epoch 2
259
total
0.10809639044479974
Epoch 3
259


In [ ]:
train()

In [ ]:
from datetime import datetime
"""
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"models/Elongation_Linear_Model.pth"
torch.save(model.state_dict(), filename)
"""

In [ ]:
model = nn.Linear(num_features, 1, bias=False)
model.load_state_dict(torch.load("models/Elongation_Linear_Model.pth"))
cuda_available = torch.cuda.is_available()
print("CUDA (GPU support) is available:", cuda_available)
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)
if cuda_available:
    """
    if num_gpus > 1:
        print("Using", num_gpus, "GPUs")
        model = torch.nn.DataParallel(model)
    """
    model = model.to('cuda')

first_param_device = next(model.parameters()).device
print("Model is on device:", first_param_device)
model.double()

In [ ]:
weights = model.weight.data.cpu().numpy()
#bias = model.bias.data.cpu().numpy()

combined = ', '.join([f'"{s}": {f}' for s, f in zip(feature_names, weights[0])])
print(combined)

#print("bias: " + str(model.bias.data))

In [ ]:
glm_kappa = [-0.0224536145637661, -0.094592589, -0.023815382, 0.030402922, -0.067234092, -0.032196914, -0.040911478, -0.018557168, -0.033545905, -0.051103287, -0.204434712, 0.015831043]

In [ ]:
"""
GLM K

* ctcf: -0.02
* h3k36me3: -0.09
* h3k4me1: -0.02
* h3k79me2: +0.03
* h3k9me1: -0.06
* h3k9me3: -0.03
* h4k20me1: -0.04
* sj5: -0.02
* sj3: -0.03
* dms->stem-loop: -0.05
* rpts->low-complex: +0.01
* wgbs->DNAm: -0.2
"""

In [ ]:
epochs = range(1, len(loss_hist_train) + 1)
plt.plot(epochs, loss_hist_train, label='train_loss')
plt.plot(epochs, loss_hist_valid, label='valid_loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show 

In [ ]:
def plot_data(glm_zeta, net_zeta):
    indices = range(len(glm_zeta))
    
    fig, ax = plt.subplots(figsize=(10, 5))
    
    ax.scatter(indices, net_zeta, color='blue', label='Neural Net Zeta', s=10, alpha=0.5)
    ax.scatter(indices, glm_zeta, color='orange', label='GLM Zeta', s=10, alpha=0.5)
    
    ax.set_title('Neural Net vs GLM Elongation Rate')
    ax.set_xlabel('Index')
    ax.set_ylabel('Elongation Rate')
    ax.legend()
    
    plt.ylim(0.5, 1.3)

    plt.show()


In [ ]:
test_batches = create_batches(test_data.groupby('GeneId'), max_batch_size=64)

test_dataset = GeneDataset(test_batches)
tstdl = DataLoader(test_dataset, batch_size=1)

In [ ]:
net_zeta = []
glm_zeta = []
with torch.no_grad():
    for batch in tstdl:
        y_inputs = batch['Y_ji'].to(device)
        rho_ji = model(y_inputs)
        # convert log(Z) outputs to Z
        net_zeta.append(torch.exp(rho_ji.cpu()[0]))
        glm_zeta.append(batch['Z_ji'][0])

net_zeta = torch.cat(net_zeta, dim=0)
glm_zeta = torch.cat(glm_zeta, dim=0)
mae = F.l1_loss(net_zeta.squeeze(), glm_zeta)
mse = F.mse_loss(net_zeta.squeeze(), glm_zeta)

correlation_coefficient = np.corrcoef(glm_zeta, net_zeta.squeeze())[0, 1]
print("Correlation Coefficient:", correlation_coefficient)

print(f"Mean Absolute Error: {mae.item():.4f}")
print(f"Mean Squared Error: {mse.item():.4f}")

In [ ]:
def density_plot(glm_zeta, net_zeta, gene_id):
    sns.kdeplot(x=glm_zeta, y=net_zeta, fill=True, cmap="Blues")
            
    plt.xlim([min(glm_zeta), max(glm_zeta)])
    plt.ylim([min(net_zeta), max(net_zeta)])


    plt.xlabel('GLM Elongation Rate')
    plt.ylabel('Neural Net Elongation Rate')
    plt.title(gene_id)
    plt.show()

In [ ]:
def scatterplot(net_zeta, glm_zeta, gene_id):
    indices = range(len(glm_zeta))
    
    fig, ax = plt.subplots(figsize=(10, 5))
    
    min_val = min(min(net_zeta), min(glm_zeta))
    max_val = max(max(net_zeta), max(glm_zeta))

    plt.xlim(min_val, max_val)
    plt.ylim(min_val, max_val)
    
    ax.scatter(net_zeta, glm_zeta, s=5)
    
    ax.set_title(gene_id)
    ax.set_xlabel('Neural Net Zeta')
    ax.set_ylabel('GLM Zeta')
    ax.legend()

    plt.show()


In [ ]:
test_batches2 = create_batches(test_data.groupby('GeneId'), max_batch_size=2000)

test_dataset2 = GeneDataset(test_batches2)
tstdl2 = DataLoader(test_dataset2, batch_size=1)

In [ ]:
total_loss = 0
loss_fn = CustomLoss()
for batch in tstdl2:
    gene_id = batch['GeneId'][0]
    model.eval()
    #print("number of samples: " + str(len(batch)))

    with torch.no_grad():
        y_inputs = batch['Y_ji'].to(device)
        rho_ji = model(y_inputs)
    
    glm_zeta = batch['Z_ji'][0]
    # convert log(Z) outputs to Z
    net_zeta = torch.exp(rho_ji.cpu().squeeze())
        
    density_plot(glm_zeta, net_zeta, gene_id)
                
    plot_data(glm_zeta, net_zeta)

In [ ]:
plt.figure(figsize=(10, 10))

sns.scatterplot(x=glm_kappa, y=weights[0])

for i in range(len(glm_kappa)):
    plt.text(glm_kappa[i], weights[0][i], feature_names[i], fontsize=13, ha='right', va='top')
plt.xlabel('GLM Weights')
plt.ylabel('Neural Net Weights')

max_val = max(np.max(glm_kappa), np.max(weights[0])) + 0.04
min_val = min(np.min(glm_kappa), np.min(weights[0])) - 0.04

plt.xlim(max_val, min_val)
plt.ylim(max_val, min_val)

# Show the plot
plt.show()

In [ ]:
# profiling code
"""
def print_profiler_results(profiler):
    print(profiler.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))

with torch.profiler.profile(
    activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
    schedule=torch.profiler.schedule(wait=1, warmup=1, active=3),
    on_trace_ready=print_profiler_results,
    record_shapes=True,
    profile_memory=True
) as profiler:

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}')
        model.train()
        trndl = DataLoader(train_set, batch_size=batch_size, num_workers=7, shuffle=False, pin_memory=True)
        for i, batch in enumerate(trndl):
            optimizer.zero_grad()
            Y_ji_batch = batch['Y_ji'].to(device)
            X_ji_batch = batch['X_ji'].to(device)
            C_j_batch = batch['C_j'].to(device)
            outputs = model(Y_ji_batch)
            loss = loss_fn(X_ji_batch, C_j_batch, outputs)
            loss.backward()
            optimizer.step()
            loss_hist_train[epoch] += loss.item()
            profiler.step()
        loss_hist_train[epoch] /= len(trndl)
        del trndl
"""

In [ ]:
# sparse loss function
"""
def sparse_dense_mul(s, d):
    i = s._indices()
    v = s._values()
    dv = d[i.squeeze()]
    return torch.sparse.FloatTensor(i, v * dv, s.size())

def sparse_dense_add(s, d):
    s = s.to(d.device)

    indices = s._indices()
    if indices.dim() == 1:
        indices = indices.squeeze()
    values = s._values()

    d[indices] += values

    return d

class SparseCustomLoss(nn.Module):
    def __init__(self):
        super(SparseCustomLoss, self).__init__()

    def forward(self, X_ji, C_j, rho_ji):
        C_j_value = C_j[0]
        X_ji = X_ji.squeeze(0)
        rho_ji = rho_ji.squeeze(0).squeeze(1)
        #X_ji_sparse = X_ji.to_sparse()
        #term1 = sparse_dense_mul(X_ji_sparse, rho_ji)
        #term2 = C_j_value * torch.exp(-rho_ji)
        #loss = sparse_dense_add(term1, term2)
        loss = X_ji * rho_ji + C_j * torch.exp(-rho_ji)
        return (loss).mean()
"""